In [13]:
from PIL import Image, ImageOps
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import os, shutil
import time
import scipy
from scipy import ndimage
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import cv2
from keras import models
from keras import layers
from sklearn.utils import class_weight

# Now lets try and to make the results better

In [14]:
# Create a data generator
datagen2 = ImageDataGenerator(
    rescale=1./255, # normalize pixel values
    rotation_range=45, # randomly rotate images in the range
    zoom_range=0.35, # randomly zoom image 
    width_shift_range=0.35, # randomly shift images horizontally
    height_shift_range=0.35, # randomly shift images vertically 
    horizontal_flip=True,
    vertical_flip=True) # randomly flip images

In [15]:
# Use flow_from_directory to read images from folders
train_generator2 = datagen2.flow_from_directory(
        'CellData/chest_xray/train', # point to the parent directory
        target_size=(299, 299), # resize images to 299x299
        batch_size=32,
        class_mode='binary') # binary for two classes

# Similarly create validation and test generators
val_generator2 = datagen2.flow_from_directory(
        'CellData/chest_xray/val',
        target_size=(299, 299),
        batch_size=32,
        class_mode='binary')

test_generator2 = datagen2.flow_from_directory(
        'CellData/chest_xray/test',
        target_size=(299, 299),
        batch_size=32,
        class_mode='binary')


Found 4098 images belonging to 2 classes.
Found 878 images belonging to 2 classes.
Found 879 images belonging to 2 classes.


In [16]:
model2 = models.Sequential()
model2.add(layers.Conv2D(64, (6, 6), activation='relu',
                        input_shape=(299 , 299,  3)))
model2.add(layers.MaxPooling2D((2, 2)))

model2.add(layers.Conv2D(64, (5, 5), activation='relu'))
model2.add(layers.MaxPooling2D((2, 2)))

model2.add(layers.Conv2D(128, (4, 4), activation='relu'))
model2.add(layers.MaxPooling2D((2, 2)))

model2.add(layers.Conv2D(256, (3, 3), activation='relu'))
model2.add(layers.MaxPooling2D((2, 2)))

model2.add(layers.Flatten())
model2.add(layers.Dense(128, activation='relu'))
model2.add(layers.Dense(256, activation='relu'))
model2.add(layers.Dense(512, activation='relu'))
model2.add(layers.Dense(1, activation='sigmoid'))

model2.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['acc'])

In [17]:
from collections import Counter
import numpy as np

# Get class distribution
counter = Counter(train_generator2.classes)                          
max_val = float(max(counter.values()))       
class_weights = {class_id : max_val/num_images for class_id, num_images in counter.items()}                     

print(counter)
print(max_val)
print(class_weights)

Counter({1: 2991, 0: 1107})
2991.0
{0: 2.7018970189701896, 1: 1.0}


In [18]:
epochs = 100

steps_per_epoch2 = np.ceil(train_generator2.samples / train_generator2.batch_size)
validation_steps2 = np.ceil(val_generator2.samples / val_generator2.batch_size)

history = model2.fit(
    train_generator2,
    steps_per_epoch=int(steps_per_epoch2),
    epochs=epochs,
    validation_data=val_generator2,
    validation_steps=int(validation_steps2),
    class_weight=class_weights)



Epoch 1/100


2023-07-13 10:21:11.397903: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


129/129 [==============================] - ETA: 0s - loss: 0.9468 - acc: 0.6176

2023-07-13 10:22:12.520113: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


129/129 [==============================] - 75s 536ms/step - loss: 0.9468 - acc: 0.6176 - val_loss: 0.7206 - val_acc: 0.5296
Epoch 2/100
129/129 [==============================] - 72s 553ms/step - loss: 0.7709 - acc: 0.7233 - val_loss: 0.6712 - val_acc: 0.6230
Epoch 3/100
129/129 [==============================] - 71s 551ms/step - loss: 0.7067 - acc: 0.7882 - val_loss: 0.3686 - val_acc: 0.8451
Epoch 4/100
129/129 [==============================] - 71s 551ms/step - loss: 0.6294 - acc: 0.8009 - val_loss: 0.3863 - val_acc: 0.8440
Epoch 5/100
129/129 [==============================] - 74s 570ms/step - loss: 0.6038 - acc: 0.8141 - val_loss: 0.3933 - val_acc: 0.8223
Epoch 6/100
129/129 [==============================] - 72s 555ms/step - loss: 0.5658 - acc: 0.8238 - val_loss: 0.3061 - val_acc: 0.8576
Epoch 7/100
129/129 [==============================] - 72s 551ms/step - loss: 0.5540 - acc: 0.8304 - val_loss: 0.3692 - val_acc: 0.8519
Epoch 8/100
129/129 [==============================] - 72s 5

Epoch 61/100
129/129 [==============================] - 72s 554ms/step - loss: 0.3679 - acc: 0.8875 - val_loss: 0.2873 - val_acc: 0.8656
Epoch 62/100
129/129 [==============================] - 71s 550ms/step - loss: 0.3720 - acc: 0.8836 - val_loss: 0.3045 - val_acc: 0.8815
Epoch 63/100
129/129 [==============================] - 72s 556ms/step - loss: 0.3599 - acc: 0.8919 - val_loss: 0.2437 - val_acc: 0.8975
Epoch 64/100
129/129 [==============================] - 71s 551ms/step - loss: 0.3661 - acc: 0.8941 - val_loss: 0.2512 - val_acc: 0.9043
Epoch 65/100
129/129 [==============================] - 72s 554ms/step - loss: 0.3675 - acc: 0.8914 - val_loss: 0.2598 - val_acc: 0.8907
Epoch 66/100
129/129 [==============================] - 72s 555ms/step - loss: 0.3531 - acc: 0.8936 - val_loss: 0.2170 - val_acc: 0.9134
Epoch 67/100
129/129 [==============================] - 71s 550ms/step - loss: 0.3596 - acc: 0.8941 - val_loss: 0.1997 - val_acc: 0.9191
Epoch 68/100
129/129 [===================

In [19]:
results_train = model2.evaluate(train_generator2)
results_test = model2.evaluate(val_generator2)
print('Training set performance:', results_train)
print('Validation set performance:', results_test)

28/28 [==============================] - 13s 454ms/step - loss: 0.1984 - acc: 0.9362
Training set performance: [0.19136229157447815, 0.9250854253768921]
Validation set performance: [0.19843034446239471, 0.9362187385559082]


In [20]:
results_train = model2.evaluate(train_generator2)
results_test = model2.evaluate(val_generator2)
print('Training set performance:', results_train)
print('Validation set performance:', results_test)

28/28 [==============================] - 13s 460ms/step - loss: 0.1859 - acc: 0.9328
Training set performance: [0.19820529222488403, 0.9192289113998413]
Validation set performance: [0.18585559725761414, 0.9328018426895142]


## worse than previous models